In [52]:
import pandas as pd
import numpy as np

# Load Data

In [58]:
# Load Hart Island Data
hart_island = pd.read_csv("https://raw.githubusercontent.com/IndigentBurials/hart-island-web-scraper/main/data/output/hart-island.csv")

# Load indigent burial data
data = pd.read_csv("./data/indigent_burials.csv")

/Users/simon/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (2,3,4,5,6,7,9,10,11,12,13,16,17,18,19,20,21,22,24,25,26,27,28,29,30,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Append new HI data to dataframe

In [59]:
# Drop all old Hart Island entries
data = data[data['Jurisdiction'] != "Hart Island"]

In [60]:
# Combine add new hart island data back in
data = pd.concat([data, hart_island])
# Reset the index
data = data.reset_index().drop(columns=['index'])

# Standardize Sex column to M/F/U

In [61]:
# Standardize column values
data['Sex'] = data['Sex'].str.upper()

# Get list of aliases from data
male_alias = ['MALE', 'M']
female_alias = ['FEMALE', 'F']
unknown_alias = ['UNKNOWN', 'UNSURE', '--']

# Create list for sex
sex_list = []

for index, row in data.iterrows():
    if row['Sex'] in male_alias:
        sex_list.append("M")
    elif row['Sex'] in female_alias:
        sex_list.append("F")
    else:
        sex_list.append("U")
        
data['Sex'] = sex_list

# Drop Age Group Column

In [62]:
data = data.drop(columns=['AgeGroup'])

# Standardize Age variable

In [70]:
# Create list for age variable
age_vals = []

# Loop through all rows and try to cast to integer
for index, row in data.iterrows():
    try:
        age = int(row['Age'])
        
        if age > 150:
            age_vals.append('NA')
        else:
            age_vals.append(age)
    except ValueError:
        age_vals.append('NA')
        
# Append new age values to dataframe
data['Age'] = age_vals
# Replace "NA" with NaN values
data['Age'] = data['Age'].replace('NA', np.nan)
# Cast the column to integer
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')

# Look into Location variable

In [76]:
data.Location.value_counts()

Fairview Cemetery             547
Tahoma                        392
Evangelico Cemetery           233
Santa Fe National Cemetery    132
Coroner's Office               34
Tahoma                          1
Name: Location, dtype: int64

# Date of Birth

In [78]:
data.DOB.value_counts()

2019-12-19    848
2019-04-23    327
2020-08-12    278
2021-10-25    206
2020-05-07    190
             ... 
2015-07-11      1
2016-11-04      1
2020-06-04      1
2020-05-16      1
2015-08-05      1
Name: DOB, Length: 1282, dtype: int64

# Create flag for gather type

In [79]:
data['GatherType'] = 'Web Scraping'

# Drop isClaimed flag

In [81]:
data = data.drop(columns=['isClaimed'])